In [1]:
%cd ..

d:\workspace\VQA


In [7]:
import pandas as pd
from config import  config
from model.vqa_model import VQAModel
from transformers import AutoTokenizer
from training import VQADataCollatorForGeneration, tokenizer
from model.processor import VQAProcessor
import torch

In [3]:
from data_processor import VQADataset
from torchvision import transforms
df_train = pd.read_csv("./dataset/csv/train_data.csv")
df_val = pd.read_csv("./dataset/csv/dev_data.csv")
df_test = pd.read_csv("./dataset/csv/test_data.csv")
transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

train_dataset = VQADataset(dataframe=df_train.loc[:5], mode="train", transform=transforms)
# val_dataset = VQADataset(dataframe=df_val, mode="val", transform=transforms)
# test_dataset = VQADataset(dataframe=df_test, mode="test", transform=transforms)


d:\workspace\VQA\data_processor.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['question'] = [word_tokenize(text_normalize(x), format='text') for x in df['question']]
d:\workspace\VQA\data_processor.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = [word_tokenize(text_normalize(str(x)), format='text') for x in df['answer']]


In [5]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
questions = ["ai là người sáng lập ra facebook", "ai là người sáng lập ra google", "ai là người sáng lập ra microsoft"]
padding = "max_length"
max_question_length = 32
truncation = True
return_tensors = "pt"

output = tokenizer(
    text=questions,
    padding=padding,
    max_length=max_question_length,
    truncation=truncation,
    return_tensors=return_tensors,
    return_attention_mask=True,
)

print(output)

{'input_ids': tensor([[    0,   277,     8,    18,   298,   740,    40,  4235,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   277,     8,    18,   298,   740,    40, 38970,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1],
        [    0,   277,     8,    18,   298,   740,    40, 13947, 36954,     2,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
data_collator = VQADataCollatorForGeneration(processor=VQAProcessor())
data_collator([train_dataset[0], train_dataset[1]])

It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


['biển ghi gì ?', 'có bao_nhiêu người đàn_ông ?']
['ghi đường hồ chí_minh', 'có hai người đàn_ông']


{'pixel_values': tensor([[[[-0.9948, -0.9948, -0.9949,  ..., -0.9976, -0.9986, -0.9997],
           [-0.9948, -0.9948, -0.9949,  ..., -0.9983, -0.9988, -0.9998],
           [-0.9947, -0.9948, -0.9949,  ..., -0.9986, -0.9992, -0.9993],
           ...,
           [-0.9942, -0.9942, -0.9943,  ..., -0.9951, -0.9976, -0.9983],
           [-0.9944, -0.9944, -0.9942,  ..., -0.9953, -0.9983, -0.9985],
           [-0.9943, -0.9942, -0.9941,  ..., -0.9949, -0.9967, -0.9974]],
 
          [[-0.9941, -0.9941, -0.9941,  ..., -0.9970, -0.9978, -0.9993],
           [-0.9941, -0.9940, -0.9940,  ..., -0.9977, -0.9981, -0.9994],
           [-0.9940, -0.9940, -0.9940,  ..., -0.9979, -0.9986, -0.9989],
           ...,
           [-0.9946, -0.9946, -0.9947,  ..., -0.9955, -0.9979, -0.9985],
           [-0.9948, -0.9948, -0.9946,  ..., -0.9956, -0.9985, -0.9985],
           [-0.9948, -0.9946, -0.9945,  ..., -0.9951, -0.9969, -0.9975]],
 
          [[-0.9939, -0.9939, -0.9939,  ..., -0.9989, -0.9993, -0.9997

In [6]:
prediction, ans_tokens = model(image, question, answer)
ans_tokens = ans_tokens.long()
print(prediction.shape)
print(ans_tokens.shape)


loss = criterion(prediction.permute(0, 2, 1), ans_tokens)
print(loss)

d:\workspace\VQA\model\vqa_model.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  h_attn = F.softmax(h_attn)


torch.Size([16, 50, 64001])
torch.Size([16, 50])
tensor(11.5250, grad_fn=<NllLoss2DBackward0>)


In [19]:
(torch.argmax(prediction, dim=2) == ans_tokens).sum().item()

0

In [9]:
tokenizer = AutoTokenizer.from_pretrained(config.text_model)

ans_decode = tokenizer.batch_decode(ans_vocab, skip_special_tokens=True)
print(ans_decode)

['họ đang đi mua_sắm trong siêu_thị', 'tỉnh thái_bình', 'có bốn người đang ở trong phòng', 'dùng cho dịch covid-19', 'cổng khu di_tích có ba lối vào', '7 700 đồng một ký', 'trên thành giếng có con_số 1554', 'trên bức tường có treo hai cái đèn_lồng', 'bán hàng', 'vì sếp tôi vừa đẹp_trai vừa tử_tế', 'dự_án khu dân_cư đông bình_dương đang triển_khai', 'bảng thông_báo san_nhượng mặt_bằng', 'một nhân_viên', 'màu đỏ và màu trắng', 'với một người phụ_nữ', 'nước_mắm này có truyền_thống trăm năm']
